In [29]:
import pandas as pd
max_words = 2000
max_len = 20
num_classes = 6

# Training
epochs = 10
batch_size = 512
print_batch_n = 100

In [7]:
!wget 'https://rusvectores.org/static/models/rusvectores4/unigrams/ruwikiruscorpora-nobigrams_upos_skipgram_300_5_2018.vec.gz'
df_train = pd.read_excel("/content/feedback.xls")
# !gunzip 'ruwikiruscorpora-nobigrams_upos_skipgram_300_5_2018.vec.gz' -d '/content/pretrained_emb'
# df_test = pd.read_csv("data/test.csv")
# df_val = pd.read_csv("data/val.csv")

gzip: /content/pretrained_emb.gz: No such file or directory


In [14]:
df_train

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
...,...,...,...
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01
20655,5,Ок,2017-06-01
20656,4,Доволен,2017-06-01
20657,1,"Песопаснасть, рут ни нужын",2017-06-01


### Предобработка

In [15]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from string import punctuation
!pip install stop_words
from stop_words import get_stop_words
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
import re
from sklearn.model_selection import train_test_split
import pandas as pd


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=74b79ede65ff1b9c23fe6bfab416d6be5b75695d23048005f2aa80fec9f91679
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words
     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 8.2 MB 7.2 MB/s 


In [16]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
# df_val['text'] = df_val['text'].apply(preprocess_text)
# df_test['text'] = df_test['text'].apply(preprocess_text)

In [17]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [18]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Отфильтруем данные

и соберём в корпус N наиболее частых токенов

In [19]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [20]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [21]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [22]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [23]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [132]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
y_train = df_train.Rating
# x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
# x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

In [31]:
x_train.shape

(20659, 20)

In [32]:
max_len

20

In [33]:
x_train[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  103,
        128,  170,  113,   77, 1451,  413,   16,  550,  449], dtype=int32)

# Keras model

In [28]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
# from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [34]:
num_classes = 6
# y_train = keras.utils.to_categorical(df_train["Rating"], num_classes)
# y_val = keras.utils.to_categorical(df_val["class"], num_classes)

In [ ]:
GlobalMaxPool1D?

In [35]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [36]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [37]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    )

Epoch 1/10
19/19 [==============================] - 4s 155ms/step - loss: 1.3649 - accuracy: 0.6874 - val_loss: 1.2436 - val_accuracy: 0.5954
Epoch 2/10
19/19 [==============================] - 3s 143ms/step - loss: 0.9392 - accuracy: 0.7231 - val_loss: 1.0674 - val_accuracy: 0.6273
Epoch 3/10
19/19 [==============================] - 3s 144ms/step - loss: 0.8122 - accuracy: 0.7427 - val_loss: 0.8938 - val_accuracy: 0.7154
Epoch 4/10
19/19 [==============================] - 3s 143ms/step - loss: 0.7132 - accuracy: 0.7657 - val_loss: 0.8398 - val_accuracy: 0.7323
Epoch 5/10
19/19 [==============================] - 3s 143ms/step - loss: 0.6610 - accuracy: 0.7772 - val_loss: 0.8014 - val_accuracy: 0.7430
Epoch 6/10
19/19 [==============================] - 3s 144ms/step - loss: 0.6230 - accuracy: 0.7857 - val_loss: 0.7811 - val_accuracy: 0.7512
Epoch 7/10
19/19 [==============================] - 3s 144ms/step - loss: 0.5941 - accuracy: 0.7919 - val_loss: 0.7774 - val_accuracy: 0.7478
Epoch 

In [57]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts([x for x in df_train.Content])
sequences = tokenizer.texts_to_sequences([x for x in df_train.Content])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 13485 unique tokens.


In [100]:
import itertools
embeddings_index = {}
f = open('/content/ruwikiruscorpora-nobigrams_upos_skipgram_300_5_2018.vec')
for line in f:
    values = line.split()
    word = values[0].split('_')[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 323348 word vectors.


In [96]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

print(take(20, embeddings_index.keys()))

['394332', 'xxxx_NUM', 'год_NOUN', 'xx_NUM', 'xxx_NUM', 'категория_PROPN', 'человек_NOUN', 'время_NOUN', 'первый_ADJ', 'мочь_VERB', 'также_ADV', 'район_NOUN', 'город_NOUN', 'становиться_VERB', 'область_NOUN', 'новый_ADJ', 'россия_PROPN', 'место_NOUN', 'являться_VERB', 'часть_NOUN']


In [102]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [104]:
embedding_matrix.shape

(13486, 300)

In [105]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            300,
                            weights=[embedding_matrix],
                            input_length=20,
                            trainable=False)

In [124]:
from keras.layers import MaxPooling1D, Conv1D, Dense, Flatten
from tensorflow.keras.utils import to_categorical
sequence_input = Input(shape=(20,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 3, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 3, activation='relu')(x)
x = MaxPooling1D(1)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(6, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model.summary()

y_train = to_categorical(y_train, 6)

model.fit(x_train, y_train, validation_split=0.2,
          epochs=epochs, batch_size=batch_size)

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 20)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 20, 300)           4045800   
_________________________________________________________________
conv1d_32 (Conv1D)           (None, 18, 128)           115328    
_________________________________________________________________
max_pooling1d_28 (MaxPooling (None, 3, 128)            0         
_________________________________________________________________
conv1d_33 (Conv1D)           (None, 1, 128)            49280     
_________________________________________________________________
max_pooling1d_29 (MaxPooling (None, 1, 128)            0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 128)               0  

In [139]:
from tensorflow.keras.layers import Embedding
sequence_input = Input(shape=(20,), dtype='int32')
embedding_layer = Embedding(max_words, 300)(sequence_input)
x = Conv1D(128, 3, activation='relu')(embedding_layer)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 3, activation='relu')(x)
x = MaxPooling1D(1)(x)
x = Flatten()(x)
x = Dense(128, activation='tanh')(x)
preds = Dense(6, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model.summary()

# y_train = to_categorical(y_train, 6)

model.fit(x_train, y_train, validation_split=0.2,
          epochs=epochs, batch_size=batch_size)


Model: "model_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_30 (InputLayer)        [(None, 20)]              0         
_________________________________________________________________
embedding_13 (Embedding)     (None, 20, 300)           600000    
_________________________________________________________________
conv1d_55 (Conv1D)           (None, 18, 128)           115328    
_________________________________________________________________
max_pooling1d_50 (MaxPooling (None, 3, 128)            0         
_________________________________________________________________
conv1d_56 (Conv1D)           (None, 1, 128)            49280     
_________________________________________________________________
max_pooling1d_51 (MaxPooling (None, 1, 128)            0         
_________________________________________________________________
flatten_23 (Flatten)         (None, 128)               0  

Сказать, что есть большой прирост точности у модели с предобученым Embedding'ом не могу. Лучше всего показала себя первая простенькая модель.